# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import argparse
import pandas as pd
import numpy as np

from recfldtkn.ckpd_obs import Ckpd_ObservationS
from recfldtkn.aidstools import get_caseset_to_observe

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/jluo/Desktop/2024-DrFirst-SPACE/_DrFirst-AI-EduRxPred-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_EduRxPred',
 'MODEL_TASK': 'Model_EduRxPred',
 'WORKSPACE_PATH': '/Users/jluo/Desktop/2024-DrFirst-SPACE/_DrFirst-AI-EduRxPred-WorkSpace'}


/Users/jluo/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Args

In [2]:

##################################
my_parser = argparse.ArgumentParser(description='Process Input.')
my_parser.add_argument('--group_id',
                    metavar='group_id',
                    default = None, 
                    type=str)

my_parser.add_argument('--caseset_name',
                    metavar='caseset_name',
                    default = None, 
                    type=str)

my_parser.add_argument('--case_id_columns',
                    metavar='case_id_columns',
                    nargs='+',
                    default = None, 
                    type=str)

my_parser.add_argument('--random_seed',
                    metavar='random_seed',
                    default = 42, 
                    type=str)

_StoreAction(option_strings=['--random_seed'], dest='random_seed', nargs=None, const=None, default=42, type=<class 'str'>, choices=None, required=False, help=None, metavar='random_seed')

In [3]:
argument_list = [
    '--group_id', '0', 
    '--caseset_name', 'TrulicityRx',
    '--case_id_columns', 'PID', 'ObsDT', 'PInvID', 'RxID', 
    '--random_seed', '42',
]

args = my_parser.parse_args(argument_list)

# Cohort and Group

In [4]:
from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import update_args_to_list

caseset_name = args.caseset_name
group_id = int(args.group_id)
RANDOM_SEED = int(args.random_seed)

# case_id_columns = update_args_to_list(args.case_id_columns)
case_id_columns = []
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['Ckpd_ObservationS'] = Ckpd_ObservationS

np.random.seed(RANDOM_SEED)

# step 2: get the case to be observed
CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder', caseset_name)
print('CaseFolder---->', CaseFolder)



group_name, ds_case = get_caseset_to_observe(group_id, CaseFolder, case_id_columns, cohort_args)
print('\n================ group_id and group_name ================')
print(group_id, group_name)
print(ds_case)

CaseFolder----> Data_EduRxPred/CaseFolder/TrulicityRx

================ group_id and group_name ================
0 0_C1-gF-Ab50
Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'DT', 'show_educational_campaigns', 'drug_name', 'ndc_id', 'cohort', 'patient_gender', 'patient_age_group', 'patient_age_bucket', 'patient_zipcode_3', 'group'],
    num_rows: 40517
})


# CaseType + RandomSeed

In [5]:
from recfldtkn.aidstools import generate_random_tags, map_case_to_IOTVT_split_type

##############
RANDOM_SEED = 42
downsample_ratio = 1 # 1 (don't drop any case), 0.1 (drop 90% of cases of one patient).
out_ratio = 0
test_ratio = '2023.11.01'#  'tail0.1'
valid_ratio = 0.1 
##############

df = ds_case.to_pandas()
print(df.shape)
print('df--->', df.shape)
df_rs = generate_random_tags(df, RANDOM_SEED)
print('df_rs--->',df_rs.shape)
df_dsmp = df_rs[df_rs['RandDownSample'] <= downsample_ratio].reset_index(drop = True)
print('df_dsmp--->', df_dsmp.shape)
df_dsmp['IOTVT'] = df_dsmp.apply(lambda x: map_case_to_IOTVT_split_type(x, out_ratio, test_ratio, valid_ratio), axis = 1)
df_dsmp['IOTVT'].value_counts().sort_index()

(40517, 14)
df---> (40517, 14)
df_rs---> (40517, 19)
df_dsmp---> (40517, 19)


IOTVT
in_test      2557
in_train    34256
in_valid     3704
Name: count, dtype: int64

In [6]:
for IOTVT, df_IOTVT in df_dsmp.groupby('IOTVT'):
    new_case_type = caseset_name + f'-rs{RANDOM_SEED}-ds{downsample_ratio}-out{out_ratio}ts{test_ratio}vd{valid_ratio}-{IOTVT}'
    path = os.path.join(SPACE['DATA_TASK'], 'CaseFolder', new_case_type, group_name + '.p')
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    print(path)
    print(IOTVT, len(df_IOTVT))
    # df_IOTVT[case_id_columns].reset_index(drop = True).to_pickle(path)
    df_IOTVT.reset_index(drop = True).to_pickle(path)

Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_test/0_C1-gF-Ab50.p
in_test 2557
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_train/0_C1-gF-Ab50.p
in_train 34256
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_valid/0_C1-gF-Ab50.p
in_valid 3704


In [7]:
df_dsmp

,PID,ObsDT,PInvID,RxID,DT,show_educational_campaigns,drug_name,ndc_id,cohort,patient_gender,patient_age_group,patient_age_bucket,patient_zipcode_3,group,RandDownSample,RandInOut,CaseRltLocInP,RandTest,RandValidation,IOTVT
0,1038447,2023-05-08 20:08:34.497,1038447-002,1038447-002-000,2023-05-08 20:08:34.497,None,Trulicity,2143380.0,C1,gF,Ab50,61-70,730.0,C1-gF-Ab50,0.000006,0.795429,0.000000,0.869130,0.996085,in_train
1,1003232,2023-09-05 16:03:59.393,1003232-000,1003232-000-000,2023-09-05 16:03:59.393,True,Trulicity,2143480.0,C1,gF,Ab50,51-60,265.0,C1-gF-Ab50,0.000012,0.866117,0.000000,0.417678,0.872480,in_train
2,1004666,2023-06-04 14:20:54.679,1004666-005,1004666-005-000,2023-06-04 14:20:54.679,None,Trulicity,2143380.0,C1,gF,Ab50,51-60,286.0,C1-gF-Ab50,0.000031,0.070227,0.000000,0.427936,0.899254,in_train
3,1024865,2023-06-03 23:51:15.065,1024865-010,1024865-010-005,2023-06-03 23:51:15.065,None,Trulicity,2223680.0,C1,gF,Ab50,71-80,763.0,C1-gF-Ab50,0.000048,0.931791,0.333333,0.769697,0.079059,in_train
4,1010548,2023-08-14 19:11:02.679,1010548-009,1010548-009-003,2023-08-14 19:11:02.679,True,Trulicity,2223601.0,C1,gF,Ab50,51-60,273.0,C1-gF-Ab50,0.000053,0.165637,0.500000,0.681837,0.773361,in_train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40512,1038463,2023-10-09 12:16:41.865,1038463-005,1038463-005-000,2023-10-09 12:16:41.865,True,Trulicity,2318280.0,C1,gF,Ab50,51-60,166.0,C1-gF-Ab50,0.999789,0.113689,0.666667,0.128496,0.660991,in_train
40513,1016942,2023-09-27 20:29:04.364,1016942-007,1016942-007-005,2023-09-27 20:29:04.364,True,Trulicity,2143380.0,C1,gF,Ab50,51-60,960.0,C1-gF-Ab50,0.999805,0.363770,0.500000,0.255392,0.520263,in_train
40514,1021388,2023-08-09 19:26:14.026,1021388-005,1021388-005-001,2023-08-09 19:26:14.026,True,Trulicity,2143380.0,C1,gF,Ab50,61-70,322.0,C1-gF-Ab50,0.999871,0.777198,0.000000,0.428076,0.720628,in_train
40515,1023831,2023-03-24 14:00:23.106,1023831-000,1023831-000-000,2023-03-24 14:00:23.106,None,Trulicity,2318280.0,C1,gF,Ab50,61-70,782.0,C1-gF-Ab50,0.999901,0.438131,0.000000,0.070497,0.091286,in_train


## Loop All Groups

In [9]:
################
print(CaseFolder)
group_name_list = list(range(100))
################

group_name_list_all = [i for i in os.listdir(CaseFolder) if '.p' in i]
group_id_to_group_name = dict(sorted({int(i.split('_')[0]): i.replace('.p', '') for i in group_name_list_all}.items()))
group_id_to_group_name_selected = {k: v for k, v in group_id_to_group_name.items() if k in group_name_list}

for group_id, group_name in group_id_to_group_name_selected.items():
    print('\n================ group_id and group_name ================')
    print(group_id, group_name)
    group_name, ds_case = get_caseset_to_observe(group_id, CaseFolder, case_id_columns, cohort_args)
    df = ds_case.to_pandas()
    df_rs = generate_random_tags(df, RANDOM_SEED)
    df_dsmp = df_rs[df_rs['RandDownSample'] <= downsample_ratio].reset_index(drop = True)
    df_dsmp['IOTVT'] = df_dsmp.apply(lambda x: map_case_to_IOTVT_split_type(x, out_ratio, test_ratio, valid_ratio), axis = 1)
    # df_dsmp['IOTVT'].value_counts()

    for IOTVT, df_IOTVT in df_dsmp.groupby('IOTVT'):
        new_case_type = caseset_name + f'-rs{RANDOM_SEED}-ds{downsample_ratio}-out{out_ratio}ts{test_ratio}vd{valid_ratio}-{IOTVT}'
        path = os.path.join(SPACE['DATA_TASK'], 'CaseFolder', new_case_type, group_name + '.p')
        if not os.path.exists(os.path.dirname(path)):
            os.makedirs(os.path.dirname(path))
        print(path)
        print(IOTVT, len(df_IOTVT))
        # df_IOTVT[case_id_columns].reset_index(drop = True).to_pickle(path)
        df_IOTVT.reset_index(drop = True).to_pickle(path)
        print(df_IOTVT.shape)


Data_EduRxPred/CaseFolder/TrulicityRx

================ group_id and group_name ================
0 0_C1-gF-Ab50
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_test/0_C1-gF-Ab50.p
in_test 2557
(2557, 20)
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_train/0_C1-gF-Ab50.p
in_train 34256
(34256, 20)
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_valid/0_C1-gF-Ab50.p
in_valid 3704
(3704, 20)

================ group_id and group_name ================
1 1_C1-gF-Bl50
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_test/1_C1-gF-Bl50.p
in_test 1107
(1107, 20)
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_train/1_C1-gF-Bl50.p
in_train 14960
(14960, 20)
Data_EduRxPred/CaseFolder/TrulicityRx-rs42-ds1-out0ts2023.11.01vd0.1-in_valid/1_C1-gF-Bl50.p
in_valid 1609
(1609, 20)

================ group_id and group_name ================
2 2_C1-gM-Ab50
Data_EduRxPred/CaseFolder/Trulic

In [10]:
df_IOTVT

,PID,ObsDT,PInvID,RxID,DT,show_educational_campaigns,drug_name,ndc_id,cohort,patient_gender,patient_age_group,patient_age_bucket,patient_zipcode_3,group,RandDownSample,RandInOut,CaseRltLocInP,RandTest,RandValidation,IOTVT
2,1027076,2023-09-24 21:14:25.074,1027076-016,1027076-016-000,2023-09-24 21:14:25.074,True,Trulicity,2143480.0,C1,gNone,Ab50,51-60,226.0,None,0.046450,0.198716,0.666667,0.929698,0.590893,in_valid
5,1045230,2023-09-20 22:39:34.469,1045230-009,1045230-009-002,2023-09-20 22:39:34.469,True,Trulicity,2143380.0,C1,gNone,Ab50,61-70,453.0,None,0.088493,0.330898,0.857143,0.971782,0.725956,in_valid
8,1022517,2023-03-31 01:13:39.386,1022517-002,1022517-002-001,2023-03-31 01:13:39.386,None,Trulicity,2143401.0,C1,gNone,Ab50,51-60,226.0,None,0.139494,0.074551,0.000000,0.907566,0.632306,in_valid
43,1045230,2023-05-30 22:52:26.160,1045230-003,1045230-003-000,2023-05-30 22:52:26.160,None,Trulicity,2143380.0,C1,gNone,Ab50,61-70,453.0,None,0.775133,0.330898,0.142857,0.942910,0.897216,in_valid
